In [ ]:
import os
import sys
import re
from pathlib import Path
from pprint import pprint

import numpy as np
from numpy.polynomial import Polynomial
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from flarepy_EK_Dra import FlareDetector

NOTEBOOK_DIR = Path().resolve()
PROJECT_ROOT = NOTEBOOK_DIR.parent

In [ ]:
print(os.getcwd()) # 現在のディレクトリを表示

In [ ]:
# "EK_Dra" フォルダーのパス。上の`print(os.getcwd())`の結果を参考に各自自分の環境に合わせて変更してください。

### pathlibというライブラリを使うことで、OSに依存しないパスの書き方に変更（パスの区切り文字を、Macなら/、Windowsなら\を自動で使い分けてくれる）
folder_path = PROJECT_ROOT / "data" / "TESS" / "EK_Dra"

# フォルダー内のファイルリスト（Pathオブジェクトとして保持）
file_list = sorted(folder_path.glob("*.fits"))

detectors = {}

for file_path in file_list:
    file_name = file_path.name

    match = re.search(r'-s\d{4}-', file_name)
    s_number = match.group(0)[1:-1] if match else None

    if not s_number:
        continue

    instance_name = f"detector_{s_number}"
    detector = FlareDetector(
        file=file_path,
        process_data=True,
        ene_thres_low=5e33,
        ene_thres_high=10e34,
    )
    detectors[instance_name] = detector

detectors = dict(sorted(detectors.items(), key=lambda x: x[1].tessBJD[0]))

for name, instance in detectors.items():
    print(f"{name}: {instance}")

In [ ]:
sum_flare_number = 0.0
sum_obs_time = 0.0
for detector in detectors.values():
    sum_flare_number += detector.flare_number
    sum_obs_time += detector.precise_obs_time
print(sum_flare_number)
print(sum_obs_time)
frequency = sum_flare_number/sum_obs_time
print(frequency)

In [ ]:
sflux = np.array([])
for  detector in detectors.values():
  sflux = np.append(
    sflux , detector.mPDCSAPflux * detector.flux_mean)
  ave = np.mean(sflux)
  print(f"mean={detector.flux_mean}")
print(ave)

In [ ]:
detectors

In [ ]:
# どれでもいいのでデータを一つ選んで、show_variables()メゾットを実行してみる。
detectors["detector_s0049"].energy
# detectors["detector_s0021"].show_variables() # これでもいい

In [ ]:
all_energy_array = []
all_precise_time = 0
for  detector in detectors.values():
    all_energy_array = np.append(
        all_energy_array , detector.energy
    )
    all_precise_time += detector.precise_obs_time
len(all_energy_array)



        # データの準備
energy_cor = np.sort(all_energy_array)  # フレアエネルギーを昇順にソート
cumenergy = np.array([len(energy_cor) - i for i in range(len(energy_cor))])

        # 累積分布のプロット
fig = go.Figure()

fig.add_trace(
                go.Scatter(
                    x=energy_cor,
                    y=cumenergy / all_precise_time,
                    mode="lines",
                    line=dict(color="gray", width=2),
                    name="All Sector",
                    line_shape="hv",  # steps-midに相当
                    )
                )

        # Adding graph title
fig.update_layout(
        #title_text=f"Flare Energy Distribution ({self.data_name})",
        title_font=dict(size=16),
        # width=800, # グラフの幅
        # height=600, # グラフの高さ
        )

# 軸の設定
fig.update_xaxes(
        title_text="Flare Energy [erg]", type="log", title_font=dict(size=15)
        )
fig.update_yaxes(
        title_text=r"Cumulative Number [day$^{-1}$]",
        type="log",
        title_font=dict(size=15),
        )

# グラフ全体の設定
fig.update_layout(
        legend=dict(x=0.05, y=0.95, font=dict(size=11)),
        # width=800,
        # height=600,
        showlegend=True,
        )

# グリッド表示
fig.update_xaxes(showgrid=True)
fig.update_yaxes(showgrid=True)

fig.show()


In [ ]:
for  detector in detectors.values():
  detector.plot_energy()

In [ ]:
detectors["detector_s0014"].plot_flare()

In [ ]:
for detector in detectors.values():
  detector.plot_flare()

In [ ]:
# plot_flare() メソッドでフレアをプロット
for  detector in detectors.values():
  detector.plot_flare()

In [ ]:
array = [1,2,3,8,15]
diff  = np.diff(array)
index1 = np.where(diff > 4)
array1_np = np.array(array)
index = array1_np + 1
print(f"{index}")

In [ ]:
import matplotlib.pyplot as plt
#2.4～3.0を1000等分して、その中のどれが自転周期なのかを探す
frequency = 1/np.linspace(1.5,5.0,10000)
total_power = np.array([])
for  detector in detectors.values():
    total_power = np.append(
        total_power, detector.power
    )
period=1/frequency
plt.plot(frequency, detectors["detector_s0015"].power)


In [ ]:
from astropy.timeseries import LombScargle
from scipy.integrate import quad
import matplotlib.pyplot as plt

x= detectors["detector_s0014"].gtessBJD
y= detectors["detector_s0014"].gmPDCSAPflux
lower_bound_fc = 0

n = len(x)
dt = 2 / (24 * 60)  # サンプリング間隔 (2 分)
frequency = np.linspace(0, 1.0 / dt, n)
power = LombScargle(x-x[0],y).power(frequency)

# 1. 周波数とPowerのグラフを描画
plt.plot(frequency, power)
plt.xlabel("Frequency")
plt.ylabel("Power")
plt.title("Frequency vs Power")
plt.grid()
plt.show()

# 2. 総面積を計算
total_area, _ = quad(lambda x: np.interp(x, frequency, power), frequency[0], frequency[-1])
print(f"Total Area: {total_area}")

# 3. 5%となる積分範囲の下限を計算
target_area = total_area * 0.05  # 面積の5%


cumulative_area = 0.0


for freq in frequency:
    cumulative_area, _ = quad(lambda x: np.interp(x, frequency, power), frequency[0], freq)
    if cumulative_area >= target_area:
        lower_bound_fc = freq
    break
print(f"Lower Bound for 5% Area: {lower_bound_fc}")

n = len(x)
dt = 2 / (24 * 60)  # サンプリング間隔 (2 分)
F = np.fft.fft(y) / (n / 2)
F[0] /= 2
F2 = F.copy()
F2[frequency > lower_bound_fc] = 0
filtered = np.fft.ifft(F2)

In [ ]:
from astropy.timeseries import LombScargle
tessBJD = detectors["detector_s0014"].gtessBJD
flux = detectors["detector_s0014"].gmPDCSAPflux

# 条件を満たすインデックスを抽出 (2 <= tessBJD <= 3)
mask = (tessBJD >= 1690.287) & (tessBJD <= 1690.356)

# 条件に合致するデータのみを抽出
filtered_tessBJD = tessBJD[mask]
filtered_flux = flux[mask]

frequency = 1/np.linspace(0.001,1,10000)
#周波数毎にどれくらい強いか(どれくらい周期的であるか)を出す
power = LombScargle(filtered_tessBJD-filtered_tessBJD[0],filtered_flux).power(frequency)

plt.plot(frequency, power)
#つまり、フレア発生部分は周波数は比べ物にならないほど高い

In [ ]:
total_bjd = np.array([])
total_flux = np.array([])*100-100
for  detector in detectors.values():
    total_bjd = np.append(
        total_bjd, detector.tessBJD
    )
    total_flux = np.append(
        total_flux, detector.mPDCSAPflux
    )

diff_bjd = np.diff(total_bjd)
gap_indices = np.where(diff_bjd >= 0.2)[0]

fig = make_subplots(rows=1, cols=1)

for i in range(len(gap_indices)):
    exclude_index = gap_indices[i]
    total_bjd[exclude_index] = np.nan
    total_flux[exclude_index] = np.nan
    fig.add_trace(
    go.Scatter(
        x=[total_bjd[exclude_index]],
        y=[total_flux[exclude_index]],
        mode="markers",
        marker=dict(color="blue", size=1),
        name="Excluded Point"
    ),
    row=1,
    col=1,
    )

fig.add_trace(
    go.Scatter(
        x=total_bjd,
        y=total_flux,
        mode="lines",
        line=dict(color="blue", width=1),
        name="Flux Line"
    ),
    row=1,
    col=1,
)

fig.update_layout(
    xaxis_range = [1850,2000],

    xaxis_title={
        'text':"時間(日)",
        'font':{
            'size':  50
        }
    },
    yaxis_title={
        'text':"平均の明るさに対する明るさ(%)",
        'font':{
            'size':  35
        }
    },
    xaxis={
        'tickfont': {
            'size': 30
        }
    },

    yaxis={
        'tickfont': {
            'size': 30
        }
    },
    showlegend=False,
    height=500,width=1700
)

fig.show()

In [ ]:
time = detectors["detector_s0049"].tessBJD
flux = detectors["detector_s0049"].mPDCSAPflux*100

diff_bjd = np.diff(detectors["detector_s0049"].tessBJD)
gap_indices = np.where(diff_bjd >= 0.2)[0]

time_excluded = time.copy()
flux_excluded = flux.copy()

fig = make_subplots(rows=1, cols=1)

for i in range(len(gap_indices)):
    exclude_index = gap_indices[i]
    time_excluded[exclude_index] = np.nan
    flux_excluded[exclude_index] = np.nan
    fig.add_trace(
    go.Scatter(
        x=[time[exclude_index]],
        y=[flux[exclude_index]],
        mode="markers",
        marker=dict(color="blue", size=1),
        name="Excluded Point"
    ),
    row=1,
    col=1,
    )

fig.add_trace(
    go.Scatter(
        x=time_excluded,
        y=flux_excluded,
        mode="lines",
        line=dict(color="blue", width=1),
        name="Flux Line"
    ),
    row=1,
    col=1,
)

fig.update_layout(

    xaxis_title={
        'text':"時間(日)",
        'font':{
            'size':  35
        }
    },
    yaxis_title={
        'text':"Normalized Flux(%)",
        'font':{
            'size':  35
        }
    },
    xaxis={
        'tickfont': {
            'size': 30
        }
    },

    yaxis={
        'tickfont': {
            'size': 30
        }
    },
    showlegend=False,
    #title=dict(text="Fig.1 Lightcurve of EK Dra", font=dict(size=40) ,x=0.5,y=0.94 ),
    height=700,width=1300
)

fig.show()


In [ ]:
time = detectors["detector_s0049"].tessBJD
flux = detectors["detector_s0049"].s2mPDCSAPflux*100

diff_bjd = np.diff(detectors["detector_s0049"].tessBJD)
gap_indices = np.where(diff_bjd >= 0.2)[0]

time_excluded = time.copy()
flux_excluded = flux.copy()

fig = make_subplots(rows=1, cols=1)

for i in range(len(gap_indices)):
    exclude_index = gap_indices[i]
    time_excluded[exclude_index] = np.nan
    flux_excluded[exclude_index] = np.nan
    fig.add_trace(
    go.Scatter(
        x=[time[exclude_index]],
        y=[flux[exclude_index]],
        mode="markers",
        marker=dict(color="blue", size=1),
        name="Excluded Point"
    ),
    row=1,
    col=1,
    )

fig.add_trace(
    go.Scatter(
        x=time_excluded,
        y=flux_excluded,
        mode="lines",
        line=dict(color="blue", width=1),
        name="Flux Line"
    ),
    row=1,
    col=1,
)

if detectors["detector_s0049"].peaktime is not None:
                for peak in detectors["detector_s0049"].peaktime:
                    fig.add_trace(
                        go.Scatter(
                            x=[peak, peak],
                            y=[1.5, 1.6],
                            mode="lines",
                            line=dict(color="red", width=1, dash="dash"),
                            showlegend=False,
                        ),
                        row=1,
                        col=1,
                    )

fig.update_layout(

    xaxis_title={
        'text':"時間(日)",
        'font':{
            'size':  35
        }
    },
    yaxis_title={
        'text':"Detrend Flux(%)",
        'font':{
            'size':  35
        }
    },
    xaxis={
        'tickfont': {
            'size': 30
        }
    },

    yaxis={
        'tickfont': {
            'size': 30
        }
    },
    showlegend=False,
    #title=dict(text="Fig.3 Detrend flux of EK Dra", font=dict(size=40) ,x=0.5,y=0.94 ),
    height=700,width=1300
)

fig.show()


In [ ]:
time = detectors["detector_s0049"].tessBJD
flux = detectors["detector_s0049"].s2mPDCSAPflux

diff_bjd = np.diff(detectors["detector_s0049"].tessBJD)
gap_indices = np.where(diff_bjd >= 0.2)[0]

time_excluded = time.copy()
detrended_flux_excluded = flux.copy()

fig = make_subplots(rows=1, cols=1)

for i in range(len(gap_indices)):
    exclude_index = gap_indices[i]
    time_excluded[exclude_index] = np.nan
    detrended_flux_excluded[exclude_index] = np.nan
    fig.add_trace(
    go.Scatter(
        x=[time[exclude_index]],
        y=[flux[exclude_index]],
        mode="markers",
        marker=dict(color="blue", size=1),
        name="Excluded Point"
    ),
    row=1,
    col=1,
    )

fig.add_trace(
    go.Scatter(
        x=time_excluded-2658.52,
        y=detrended_flux_excluded,
        mode="lines",
        line=dict(color="blue", width=7),
        name="Flux Line"
    ),
    row=1,
    col=1,
)



# if detectors["detector_s0049"].peaktime is not None:
#                 for peak in detectors["detector_s0049"].peaktime:
#                     fig.add_trace(
#                         go.Scatter(
#                             x=[peak, peak],
#                             y=[0.013, 0.0135],
#                             mode="lines",
#                             line=dict(color="red", width=5, dash="dash"),
#                             showlegend=False,
#                         ),
#                         row=1,
#                         col=1,
#                     )
fig.update_layout(
    # xaxis_title={
    #     'text':"時間(日)",
    #     'font':{
    #         'size':  50
    #     }
    # },
    # yaxis_title={
    #     'text':"明るさ",
    #     'font':{
    #         'size':  50
    #     }
    # },
    xaxis_range=[0,0.14],
    xaxis={
        'tickfont': {
            'size': 30
        }
    },

    yaxis={
        'tickfont': {
            'size': 30
        }
    },
    showlegend=False,
    title=dict(text="Fig.2 Close-up of Flare Event ", font=dict(size=60) ,x=0.5, y=0.96 ),
    height=700,width=1300
)
fig.show()

In [ ]:
# plot_energy() メソッドでエネルギーをプロット
for  detector in detectors.values():
  detector.plot_flare()
  detector.plot_energy()

In [ ]:
# 気になった値について、plotly_expressを使ってプロットする
# 配列の要素数に注意する必要が多分ある。
# s2mPDCSAPflux をプロット
for name, instance in detectors.items():
  # print(instance.tessBJD)
  # print(instance.s2mPDCSAPflux)
  fig = px.scatter(
    x=instance.tessBJD,
    y=instance.s2mPDCSAPflux,
    title=f"TESS BJD vs PDCSAP Flux ({name})",
    labels={"x": "TESS BJD", "y": "PDCSAP Flux"}
  )
  fig.update_traces(marker=dict(size=1), name="Flux Data")
  fig.update_layout(showlegend=True)
  fig.show()

In [ ]:
# クラス変数は`クラス名.変数名`でアクセスできる
# print(f"フレア検出割合の配列{FlareDetector.array_flare_ratio}")
# print(f"観測時間の配列{FlareDetector.array_observation_time}")

fig = px.scatter(
  x=FlareDetector.array_observation_time,
  y=FlareDetector.array_flare_ratio,
  title=f"フレア検出割合の時間変化",
  labels={"x": "観測時間", "y": "フレア検出割合"},
  hover_data={"データ": list(FlareDetector.array_data_name)} # ホバーした時にデータ名が表示されるようにする
)
fig.update_traces(marker=dict(size=10), name="Flare Ratio")
fig.update_layout(showlegend=True)
fig.show()
# print(detectors["detector_s0021"].tessBJD)

In [ ]:
fig = px.scatter(
  x=FlareDetector.array_observation_time,
  y=FlareDetector.array_amplitude,
  title=f"明るさの振幅の時間変化",
  labels={"x": "観測時間", "y": "明るさの振幅"}
)

fig.update_traces(marker=dict(size=10), name="Flare Amplitude")
fig.update_layout(showlegend=True)
fig.show()

In [ ]:
# 簡易版
# px.scatter(x=FlareDetector.array_amplitude, y=FlareDetector.array_flare_ratio).show()

fig = px.scatter(
  x=FlareDetector.array_amplitude,
  y=FlareDetector.array_flare_ratio,
  labels={"x": "明るさの振幅(黒点面積)", "y": "フレアの数/日"},
  hover_data={"データ": list(FlareDetector.array_data_name)} # ホバーした時にデータ名が表示されるようにする
)

#フィッティング
coeffs = np.polyfit(FlareDetector.array_amplitude,FlareDetector.array_flare_ratio,1)
linear_fit = np.poly1d(coeffs)

fit_y = linear_fit(FlareDetector.array_amplitude)

fig.add_scatter(x=FlareDetector.array_amplitude, y=fit_y, mode='lines', name='線形フィッティング線', line=dict(width=5))

fig.update_traces(marker=dict(size=10), name="Flare Amplitude")
fig.update_layout(
    xaxis_title={
        'text':"明るさの変動(黒点面積)",
        'font':{
            'size':  40
        }
    },
    yaxis_title={
        'text':"フレアの数/日(個)",
        'font':{
            'size':  40
        }
    },
    xaxis={
        'tickfont': {
            'size': 25
        }
    },

    yaxis={
        'tickfont': {
            'size': 30
        }
    },
    showlegend=False,
    title=dict(text="図3.黒点面積とフレアの発生頻度の関係", font=dict(size=35) ,x=0.5 , y=0.97),
    height=700,width=1000
  )
fig.show()

In [ ]:
fig = px.scatter(
  x=FlareDetector.array_observation_time,
  y=FlareDetector.array_per,
  title=f"自転周期の時間変化",
  labels={"x": "観測時間", "y": "自転の周期"},
  hover_data={"データ": list(FlareDetector.array_data_name)} # ホバーした時にデータ名が表示されるようにする
)
#誤差バーを追加
fig.update_traces(
    marker=dict(size=10),
    name="Flare Amplitude",
    error_y=dict(
        type='data',
        array=FlareDetector.array_per_err,
        visible=True
    )
)
#フィッティング
def poly_fit(x,coeffs):
    p= Polynomial(coeffs)
    return p(x)

poly_coeff = Polynomial.fit(FlareDetector.array_observation_time, FlareDetector.array_per , 3).convert().coef

fit_y = poly_fit(FlareDetector.array_observation_time, poly_coeff)

fig.add_scatter(x=FlareDetector.array_observation_time, y=fit_y, mode='lines', name='フィッティング線')

fig.update_traces(marker=dict(size=10), name="Flare Amplitude")
fig.update_layout(showlegend=True, height=700,width=1000)
fig.show()

In [ ]:
fig = px.scatter(
  x=FlareDetector.array_amplitude,
  y=FlareDetector.array_energy_ratio,
  title=f"黒点面積とフレアエネルギー",
  labels={"x": "黒点面積", "y": "フレアのエネルギーの合計"}
)

coeffs = np.polyfit(FlareDetector.array_amplitude,FlareDetector.array_energy_ratio,1)
linear_fit = np.poly1d(coeffs)

fit_y = linear_fit(FlareDetector.array_amplitude)

fig.add_scatter(x=FlareDetector.array_amplitude, y=fit_y, mode='lines', name='線形フィッティング線', line=dict(width=5))

fig.update_traces(marker=dict(size=10), name="Flare Amplitude")
fig.update_layout(
    xaxis_title={
        'text':"明るさの変動(黒点面積)",
        'font':{
            'size':  40
        }
    },
    yaxis_title={
        'text':"フレアのエネルギーの合計(erg/日)",
        'font':{
            'size':  40
        }
    },
    xaxis={
        'tickfont': {
            'size': 25
        }
    },

    yaxis={
        'tickfont': {
            'size': 30
        }
    },
    showlegend=False,
    #title=dict(text="図3.黒点面積とフレアの発生頻度の関係", font=dict(size=35) ,x=0.5 , y=0.97),
    height=700,width=1000
  )

fig.update_traces(marker=dict(size=10), name="Flare Amplitude")
fig.update_layout(showlegend=True, height=700,width=1000)
fig.show()

In [ ]:
fig = px.scatter(
  x=FlareDetector.array_amplitude,
  y=FlareDetector.array_energy_ratio/FlareDetector.array_flare_ratio,
  title=f"黒点面積とフレアエネルギー",
  labels={"x": "黒点面積", "y": "フレア1つあたりのエネルギー"}
)

coeffs = np.polyfit(FlareDetector.array_amplitude,FlareDetector.array_energy_ratio/FlareDetector.array_flare_ratio,1)
linear_fit = np.poly1d(coeffs)

fit_y = linear_fit(FlareDetector.array_amplitude)

fig.add_scatter(x=FlareDetector.array_amplitude, y=fit_y, mode='lines', name='線形フィッティング線', line=dict(width=5))

fig.update_traces(marker=dict(size=10), name="Flare Amplitude")
fig.update_layout(
    xaxis_title={
        'text':"明るさの変動(黒点面積)",
        'font':{
            'size':  40
        }
    },
    yaxis_title={
        'text':"フレア1つあたりのエネルギー(erg)",
        'font':{
            'size':  40
        }
    },
    xaxis={
        'tickfont': {
            'size': 25
        }
    },

    yaxis={
        'tickfont': {
            'size': 30
        }
    },
    showlegend=False,
    #title=dict(text="図3.黒点面積とフレアの発生頻度の関係", font=dict(size=35) ,x=0.5 , y=0.97),
    height=700,width=1000
  )

fig.update_traces(marker=dict(size=10), name="Flare Amplitude")
fig.update_layout(showlegend=True, height=700,width=1000)
fig.show()
print(f"{FlareDetector.array_energy_ratio}/{FlareDetector.array_flare_ratio}={FlareDetector.array_energy_ratio/FlareDetector.array_flare_ratio}")

In [ ]:
fig = px.scatter(
  x=FlareDetector.array_flare_ratio,
  y=FlareDetector.array_energy_ratio,
  title=f"フレア頻度とフレアエネルギー",
  labels={"x": "フレア頻度", "y": "フレアのエネルギーの合計"}
)

coeffs = np.polyfit(FlareDetector.array_flare_ratio,FlareDetector.array_energy_ratio,1)
linear_fit = np.poly1d(coeffs)

fit_y = linear_fit(FlareDetector.array_flare_ratio)

fig.add_scatter(x=FlareDetector.array_flare_ratio, y=fit_y, mode='lines', name='線形フィッティング線', line=dict(width=5))

fig.update_traces(marker=dict(size=10), name="Flare Amplitude")
fig.update_layout(
    xaxis_title={
        'text':"フレア頻度",
        'font':{
            'size':  40
        }
    },
    yaxis_title={
        'text':"フレアのエネルギーの合計(erg/日)",
        'font':{
            'size':  40
        }
    },
    xaxis={
        'tickfont': {
            'size': 25
        }
    },

    yaxis={
        'tickfont': {
            'size': 30
        }
    },
    showlegend=False,
    #title=dict(text="図3.黒点面積とフレアの発生頻度の関係", font=dict(size=35) ,x=0.5 , y=0.97),
    height=700,width=1000
  )

fig.update_traces(marker=dict(size=10), name="Flare Amplitude")
fig.update_layout(showlegend=True, height=700,width=1000)
fig.show()

In [ ]:
fig = px.scatter(
  x=FlareDetector.array_per,
  y=FlareDetector.array_flare_ratio,
  title=f"自転周期とフレアの数",
  labels={"x": "自転周期", "y": "フレアの数"}
)

fig.update_traces(marker=dict(size=10), name="Flare Amplitude")
fig.update_layout(showlegend=True, height=700,width=1000)
fig.show()

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['xtick.major.width'] = 1.5#x軸主目盛り線の線幅
plt.rcParams['ytick.major.width'] = 1.5#y軸主目盛り線の線幅
plt.rcParams['axes.linewidth'] = 1.5# 軸の線幅edge linewidth。囲みの太さ
plt.rcParams['xtick.major.size'] = 7
plt.rcParams['ytick.major.size'] = 7
plt.rcParams["xtick.minor.visible"]=True
plt.rcParams["ytick.minor.visible"]=True
plt.rcParams['xtick.minor.width'] = 1.5#x軸主目盛り線の線幅
plt.rcParams['ytick.minor.width'] = 1.5#y軸主目盛り線の線幅
plt.rcParams['xtick.minor.size'] = 4
plt.rcParams['ytick.minor.size'] = 4
plt.rcParams['xtick.direction'] = 'in' # x axis in
plt.rcParams['ytick.direction'] = 'in' # y axis in
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
fig, ax = plt.subplots(figsize=(8, 6))
plt.errorbar(FlareDetector.array_per, FlareDetector.array_starspot/1e17, yerr = None, xerr = FlareDetector.array_per_err, marker="o" ,color="black", ls="none", ms=6, capsize=4, zorder=1, mew=1.5)
plt.plot( FlareDetector.array_per, FlareDetector.array_starspot/1e17, zorder=0, color="black", ls="none")
plt.xlabel(f'Rotation Period [day] ', fontsize=20)
plt.ylabel(r"Starspot Area [10$^{21}$ cm$^{2}$]", size=20)
plt.xlim([2.3, 3.1])
plt.ylim([0.1, 2.2])
plt.tick_params(labelsize=16)
leg = plt.legend(loc='upper right', fontsize=16)
leg.get_frame().set_alpha(0)  # 背景を完全に透明にする
plt.savefig(NOTEBOOK_DIR/'period_vs_spotarea.pdf', format='pdf', bbox_inches='tight')
plt.show()

In [ ]:
fig = px.scatter(
  x=FlareDetector.array_per,
  y=FlareDetector.array_starspot,
  labels={"x": "自転周期", "y": "明るさの振幅(黒点面積)"}
)
fig.update_traces(
    marker=dict(size=10),
    name="Flare Amplitude",
    error_x=dict(
        type='data',
        array=FlareDetector.array_per_err,
        visible=True,
        color="blue",
        thickness=1
    )
)

fig.update_traces(marker=dict(size=10,color="red"), name="Flare Amplitude")
fig.update_layout(
    xaxis_title={
        'text':"自転周期(日)",
        'font':{
            'size':  35
        }
    },
    yaxis_title={
        'text':"黒点面積(m^2)",
        'font':{
            'size':  35
        }
    },
    xaxis={
        'tickfont': {
            'size': 30
        }
    },

    yaxis={
        'tickfont': {
            'size': 30
        }
    },
    showlegend=False,
    #title=dict(text="Fig.5 Rotation Period vs Starspot Area", font=dict(size=40) ,x=0.5,y=0.97 ),
    height=700,width=1300
)
fig.show()


In [ ]:
energy_ave = [a/b for a,b in zip(FlareDetector.array_energy_ratio,FlareDetector.array_flare_ratio)]
fig = px.scatter(
  x=FlareDetector.array_amplitude,
  y=energy_ave,
  labels={"x": "明るさの振幅(黒点面積)", "y": "フレアの数/日"},
  hover_data={"データ": list(FlareDetector.array_data_name)} # ホバーした時にデータ名が表示されるようにする
)

#フィッティング
coeffs = np.polyfit(FlareDetector.array_amplitude,energy_ave,1)
linear_fit = np.poly1d(coeffs)

fit_y = linear_fit(FlareDetector.array_amplitude)

fig.add_scatter(x=FlareDetector.array_amplitude, y=fit_y, mode='lines', name='線形フィッティング線')

fig.update_traces(marker=dict(size=10), name="Flare Amplitude")
fig.update_layout(
    xaxis_title={
        'text':"明るさの変動(黒点面積)",
        'font':{
            'size':  50
        }
    },
    yaxis_title={
        'text':"フレアの平均エネルギー",
        'font':{
            'size':  50
        }
    },
    xaxis={
        'tickfont': {
            'size': 25
        }
    },

    yaxis={
        'tickfont': {
            'size': 30
        }
    },
    showlegend=False,
    height=700,width=1000
  )
fig.show()

In [ ]:
# energy_ave = [a/b for a,b in zip(FlareDetector.array_energy_ratio,FlareDetector.array_flare_ratio)]
# energy_ave[8] = np.nan
# FlareDetector.array_amplitude[8] = np.nan
# amplitude_clean = []
# energy_clean = []
# data_name_clean = []

# for i in range(len(FlareDetector.array_amplitude)):
#     if not np.isnan(FlareDetector.array_amplitude[i]) and not np.isnan(energy_ave[i]):
#         amplitude_clean.append(FlareDetector.array_amplitude[i])
#         energy_clean.append(energy_ave[i])
#         data_name_clean.append(FlareDetector.array_data_name[i])
# fig = px.scatter(
#   x=amplitude_clean,
#   y=energy_clean,
#   labels={"x": "明るさの振幅(黒点面積)", "y": "フレアの数/日"},
#   hover_data={"データ": list(data_name_clean)} # ホバーした時にデータ名が表示されるようにする
# )

# #フィッティング
# coeffs = np.polyfit(amplitude_clean,energy_clean,1)
# linear_fit = np.poly1d(coeffs)

# fit_y = linear_fit(amplitude_clean)

# fig.add_scatter(x=amplitude_clean, y=fit_y, mode='lines', name='線形フィッティング線')

# fig.update_traces(marker=dict(size=10), name="Flare Amplitude")
# fig.update_layout(
#     yaxis_range = [5e33,5e34],
#     xaxis_title={
#         'text':"明るさの振幅(黒点面積)",
#         'font':{
#             'size':  50
#         }
#     },
#     yaxis_title={
#         'text':"フレアの平均エネルギー",
#         'font':{
#             'size':  50
#         }
#     },
#     xaxis={
#         'tickfont': {
#             'size': 25
#         }
#     },

#     yaxis={
#         'tickfont': {
#             'size': 30
#         }
#     },
#     showlegend=False,
#     height=700,width=1000
#   )
# fig.show()

In [ ]:
max_energy = np.array([])
for  detector in detectors.values():
    energy_cor=np.sort(detector.energy)
    max_energy = np.append(
        max_energy, energy_cor[-1]
    )
    max_energy = np.append(
        max_energy, energy_cor[-2]
    )
amplitudes = np.repeat(FlareDetector.array_amplitude, 2)
fig = px.scatter(
  x=amplitudes,
  y=max_energy,
  labels={"x": "明るさの振幅(黒点面積)", "y": "top2フレアのエネルギー"},
  #hover_data={"データ": list(FlareDetector.array_data_name)} # ホバーした時にデータ名が表示されるようにする
)
fig.update_traces(marker=dict(size=10), name="Flare Energy")
fig.update_layout(showlegend=True)
fig.show()

In [ ]:
max_energy = np.array([])
for  detector in detectors.values():
    energy_cor=np.sort(detector.energy)
    ave_max2_ene = (energy_cor[-1]+energy_cor[-2])/2
    max_energy = np.append(
        max_energy, ave_max2_ene
    )
fig = px.scatter(
  x=FlareDetector.array_amplitude,
  y=max_energy,
  labels={"x": "明るさの振幅(黒点面積)", "y": "top2フレアの平均エネルギー"},
  hover_data={"データ": list(FlareDetector.array_data_name)} # ホバーした時にデータ名が表示されるようにする
)
fig.update_traces(marker=dict(size=10), name="Flare Energy")
fig.update_layout(showlegend=True)
fig.show()

In [ ]:
max_energy = np.array([])
for  detector in detectors.values():
    energy_cor=np.sort(detector.energy)
    max_energy = np.append(
        max_energy, energy_cor[-1]
    )
fig = px.scatter(
  x=FlareDetector.array_amplitude,
  y=max_energy,
  labels={"x": "明るさの振幅(黒点面積)", "y": "フレアの最大エネルギー"},
  hover_data={"データ": list(FlareDetector.array_data_name)} # ホバーした時にデータ名が表示されるようにする
)
fig.update_traces(marker=dict(size=10), name="Flare Energy")
fig.update_layout(showlegend=True)
fig.show()


In [ ]:
min_energy = np.array([])
for  detector in detectors.values():
    energy_cor=np.sort(detector.energy)
    min_energy = np.append(
        min_energy, energy_cor[0]
    )
fig = px.scatter(
  x=FlareDetector.array_amplitude,
  y=min_energy,
  labels={"x": "明るさの振幅(黒点面積)", "y": "フレアの最小エネルギー"},
  hover_data={"データ": list(FlareDetector.array_data_name)} # ホバーした時にデータ名が表示されるようにする
)
fig.update_traces(marker=dict(size=10), name="Flare Energy")
fig.update_layout(showlegend=True)
fig.show()


In [ ]:
min_energy = np.array([])
for  detector in detectors.values():
    energy_cor=np.sort(detector.energy)
    min_energy = np.append(
        min_energy, energy_cor[0]
    )
max_energy = np.array([])
for  detector in detectors.values():
    energy_cor=np.sort(detector.energy)
    max_energy = np.append(
        max_energy, energy_cor[-1]
    )
fig = px.scatter(
  x=max_energy,
  y=min_energy,
  labels={"x": "フレアの最大エネルギー", "y": "フレアの最小エネルギー"},
  hover_data={"データ": list(FlareDetector.array_data_name)} # ホバーした時にデータ名が表示されるようにする
)

coeffs = np.polyfit(max_energy,min_energy,1)
linear_fit = np.poly1d(coeffs)

fit_y = linear_fit(max_energy)

fig.add_scatter(x=max_energy, y=fit_y, mode='lines', name='線形フィッティング線')

fig.update_traces(marker=dict(size=10), name="Flare Energy")
fig.update_layout(showlegend=True, height=1000,width=1000)
fig.show()

In [ ]:
detectors["detector_s0016"].sum_flare_energy

### 作るの遅れてごめん。わからんとこあったらいつでも聞いて〜

あとは、前大治郎が言ってたみたいに、明らかにミスって検出しているところを修正できるように flarepy_improved.py 内の関数を修正したい。
